# Decision Tree & Random Forest

## What is a decision tree?
for example
![](https://i.imgur.com/thvDsjB.png)
Each **non-leaf node**(blue triangle) in this tree is basically a decision maker. <br>
These nodes are called **decision nodes**. <br>
Each node carries out a specific test to determine where to go next. <br>
Depending on the outcome, you either go to the left branch or the right branch of this node. <br>
We keep doing this until we reach a **leaf node**（pink oval）. <br>
If we are constructing a classifier, each leaf node represents a **class**. <br>

## Types of Decision Tree 
use different measure to find the best split point. 

* **ID3**
    * entropy (the smaller the better)
    * information gain (the bigger the better)
    ![](https://i.imgur.com/PjFEJxD.png)
    ![](https://i.imgur.com/MDYGhxP.png)
* **C4.5**
    * improve ID3
    * gain ratio
    ![](https://i.imgur.com/gjxKWBn.png)
    ![](https://i.imgur.com/S2pLnOG.png)
* **CART**
    * binary tree
    * gini index (the smaller the better)
    

## Implement Decision Tree using CART  
(**C**lassification **A**nd **R**egression **T**ree algorithm) <br><br>
CART use **gini index** to find the best **split point**. <br>
Gini **impurity** score is a measure of how the data would be **incorrectly** labeled. <br>
* Gini = **0.0** -> **best** (i.e. when we use this attribute to classify the dataset, it can be clearly catrgorized.)<br>
* Gini = **0.5** -> **worst** (i.e. this attribute is not an important feature to make a decision.)<br>
![](https://i.imgur.com/FDrWorX.png)

![](https://i.imgur.com/pkPtylp.png)

    
## import packages

In [32]:
from random import randrange
from csv import reader
from math import sqrt

## Data Loading and Preprocessing 
convert string attributes and class column to integers.

In [33]:
def load_csv(filename):
    dataset = list()
    with open(filename, 'r') as file:
        csv_reader = reader(file)
        for row in csv_reader:
            if not row:
                continue
            #add that row as an element in the dataset list (2D Matrix of values)
            dataset.append(row)
    return dataset
 
def str_column_to_float(dataset, column):
    for row in dataset:
        row[column] = float(row[column].strip())

def str_column_to_int(dataset, column):
    class_values = [row[column] for row in dataset]
    unique = set(class_values)
    lookup = dict()
    for i, value in enumerate(unique):
        lookup[value] = i
    for row in dataset:
        row[column] = lookup[row[column]]
    return lookup


filename = 'test.csv'
dataset = load_csv(filename)

# convert string attributes to integers
for i in range(0, len(dataset[0])-1):
    str_column_to_float(dataset, i)
    
# convert class column to integers
str_column_to_int(dataset, len(dataset[0])-1)

dataset

[[1.0, 1.0, 0],
 [1.0, 1.0, 0],
 [1.0, 0.0, 1],
 [0.0, 1.0, 1],
 [0.0, 1.0, 1],
 [1.0, 1.0, 0],
 [1.0, 1.0, 0],
 [1.0, 0.0, 1],
 [0.0, 1.0, 1],
 [0.0, 1.0, 1]]

## Decision Tree
use **gini** to select the best **split point** for the dataset and recursively construct the tree.

In [34]:
# Calculate gini impurity
def gini_index(groups, class_values):
    gini = 0.0
    for class_value in class_values:
        for group in groups:
            size = len(group)
            if size == 0:
                continue
                
            ############################## gini formulation ##############################
            proportion = [row[-1] for row in group].count(class_value) / float(size)
            gini += (proportion * (1.0 - proportion))
            ##############################################################################
            
    return gini

 
# Calculate the best split point
def get_split(dataset, n_features):
    class_values = list(set(row[-1] for row in dataset))
    b_index, b_value, b_score, b_groups = 999, 999, 999, None
    features = list()
    while len(features) < n_features:
        index = randrange(len(dataset[0])-1)
        if index not in features:
            features.append(index)
    for index in features:
        for row in dataset:
            groups = test_split(index, row[index], dataset)
            gini = gini_index(groups, class_values)
            if gini < b_score:
                b_index, b_value, b_score, b_groups = index, row[index], gini, groups

    return {'index':b_index, 'value':b_value, 'groups':b_groups}
 

# Create a terminal node value
def to_terminal(group):
    #select a class value for a group of rows
    outcomes = [row[-1] for row in group]
    #returns the most common output value in a list of rows
    return max(set(outcomes), key=outcomes.count)
 
#Create child splits for a node or make terminal
def split(node, max_depth, min_size, n_features, depth):
    left, right = node['groups']
    del(node['groups'])
    
    # check for a no split
    if not left or not right:
        # create a terminal node
        node['left'] = node['right'] = to_terminal(left + right)
        return
    
    # check for max depth
    if depth >= max_depth:
        # create a terminal node
        node['left'], node['right'] = to_terminal(left), to_terminal(right)
        return
    
    # process the left child
    # check if the group of rows is too small
    if len(left) <= min_size:
        # create a terminal node
        node['left'] = to_terminal(left)
    else:
        # create and add the left node in a depth first fashion until the bottom of the tree is reached on this branch.
        node['left'] = get_split(left, n_features)
        split(node['left'], max_depth, min_size, n_features, depth+1)
        
    # process right child
    if len(right) <= min_size:
        node['right'] = to_terminal(right)
    else:
        node['right'] = get_split(right, n_features)
        split(node['right'], max_depth, min_size, n_features, depth+1)

        
def build_tree(train, max_depth, min_size, n_features):
    #create root
    root = get_split(train, n_features)
    #call the split function recursively to build out the whole tree
    split(root, max_depth, min_size, n_features, 1)
    return root

### Evaluation
a single subsample is retained as the **validation data**. <br>
for testing the model, and the remaining k − 1 subsamples are used as **training data**. <br>
The cross-validation process is then repeated k times (the folds), <br>
with each of the k subsamples used exactly **once** as the validation data.

In [35]:
def cross_validation_split(dataset, n_folds):
    dataset_split = list()
    dataset_copy = list(dataset)
    fold_size = int(len(dataset) / n_folds)
    for i in range(n_folds):
        fold = list()
        while len(fold) < fold_size:
            index = randrange(len(dataset_copy))
            fold.append(dataset_copy.pop(index))
        dataset_split.append(fold)
    return dataset_split

# Split a dataset based on an attribute and an attribute value
def test_split(index, value, dataset):
    left, right = list(), list()
    for row in dataset:
        #if the value at that row is less than the given value
        if row[index] < value:
            left.append(row)
        else:
            right.append(row)
    return left, right
 
# Calculate accuracy percentage
def accuracy_metric(actual, predicted):
    correct = 0  #correct predictions
    for i in range(len(actual)):
        if actual[i] == predicted[i]:
            correct += 1
    return correct / float(len(actual)) * 100.0
 
# Evaluate an algorithm using cross validation 
def evaluate_algorithm(dataset, algorithm, n_folds, *args):
    folds = cross_validation_split(dataset, n_folds)
    scores = list()
    for fold in folds:
        train_set = list(folds)
        #remove the given subsample
        train_set.remove(fold)
        train_set = sum(train_set, [])
        test_set = list()
        #add each row in a given subsample to the test set
        for row in fold:
            row_copy = list(row)
            test_set.append(row_copy)
            # remove answer
            row_copy[-1] = None
        
        predicted = algorithm(train_set, test_set, *args)  #get predicted labls
        actual = [row[-1] for row in fold]  #get actual labels
        accuracy = accuracy_metric(actual, predicted)
        scores.append(accuracy)
    return scores

### Prediction
When **validating** and **testing**, we need to predict answers for the validation set and testing set according to the trees based on the training set. <br>
We must check if a child node is :
* a **terminal** value to be returned as the prediction 
* a **dictionary** node containing another level of the tree to be considered

In [36]:
def predict(node, row):
    if row[node['index']] < node['value']:
        if isinstance(node['left'], dict):
            return predict(node['left'], row)
        else:
            return node['left']
    else:
        if isinstance(node['right'], dict):
            return predict(node['right'], row)
        else:
            return node['right']

## Random Forest
1. Create **k** bootstrap samples.
2. Train a classifier on each bootstrap sample.
3. Vote (or average) the predictions of **k** models.

![](https://i.imgur.com/Re2f26x.png)

## Multiple Decision Trees -> Forest
The original dataset is **randomly partitioned** into k equal sized subsamples. <br>
We use every subsample to build different decision trees.
![](https://i.imgur.com/l8B4Y85.png)

In [37]:
def subsample(dataset, ratio):
    sample = list()
    n_sample = round(len(dataset) * ratio)
    while len(sample) < n_sample:
        index = randrange(len(dataset))
        sample.append(dataset[index])
    return sample

## Bagging
(**B**ootstrap **agg**regat**ing**) <br>
Select the most **common** prediction from the list of predictions made by the **bagged trees**.

In [38]:
def bagging_predict(trees, row):
    predictions = [predict(tree, row) for tree in trees]
    # list.count count the occurences of "0"."1"
    return max(set(predictions), key=predictions.count)
 
def random_forest(train, test, max_depth, min_size, sample_size, n_trees, n_features):
    trees = list()  # store the forest
    # generate trees
    for i in range(n_trees):
        ######################### split original dataset into subsamples #####################
        ######################### and build trees for every subsample ########################
        sample = subsample(train, sample_size)
        tree = build_tree(sample, max_depth, min_size, n_features)
        ######################################################################################
        print ("chosen sample:",sample)
        print ("Decision Tree:",tree) 
        trees.append(tree)
    predictions = [bagging_predict(trees, row) for row in test]
    print ("test data:",test)
    print ("prediction:",predictions)
    print ("--")
    return predictions
 


# you can define your own parameters
n_folds = 5  # number of subsamples
max_depth = 10   # max height of every decision tree
min_size = 1    # minimum sample to generate a node
sample_size = 1.0   # ratio 
n_features = int(sqrt(len(dataset[0])-1))  # number of features

# you can choose the number of trees in your forest
for n_trees in [1, 5]:
    scores = evaluate_algorithm(dataset, random_forest, n_folds, max_depth, min_size, sample_size, n_trees, n_features)
    print('Trees: %d' % n_trees)
    print('Scores: %s' % scores)
    print('Mean Accuracy: %.3f%%' % (sum(scores)/float(len(scores))))
    print('---------------------------------------------------------------------------------------------')
    

chosen sample: [[0.0, 1.0, 1], [1.0, 1.0, 0], [0.0, 1.0, 1], [1.0, 1.0, 0], [1.0, 0.0, 1], [1.0, 1.0, 0], [1.0, 1.0, 0], [1.0, 0.0, 1]]
Decision Tree: {'index': 0, 'value': 1.0, 'left': {'index': 0, 'value': 0.0, 'left': 1, 'right': 1}, 'right': {'index': 1, 'value': 1.0, 'left': {'index': 0, 'value': 1.0, 'left': 1, 'right': 1}, 'right': {'index': 1, 'value': 1.0, 'left': 0, 'right': 0}}}
test data: [[1.0, 0.0, None], [0.0, 1.0, None]]
prediction: [1, 1]
--
chosen sample: [[0.0, 1.0, 1], [1.0, 1.0, 0], [1.0, 1.0, 0], [1.0, 1.0, 0], [1.0, 1.0, 0], [1.0, 1.0, 0], [0.0, 1.0, 1], [1.0, 1.0, 0]]
Decision Tree: {'index': 1, 'value': 1.0, 'left': 0, 'right': 0}
test data: [[1.0, 1.0, None], [1.0, 1.0, None]]
prediction: [0, 0]
--
chosen sample: [[1.0, 0.0, 1], [1.0, 0.0, 1], [1.0, 1.0, 0], [1.0, 1.0, 0], [1.0, 1.0, 0], [1.0, 0.0, 1], [0.0, 1.0, 1], [1.0, 1.0, 0]]
Decision Tree: {'index': 1, 'value': 1.0, 'left': {'index': 0, 'value': 1.0, 'left': 1, 'right': 1}, 'right': {'index': 1, 'value'